In [1]:
# Importing Dependencies
import numpy as np
import gym
from gym.envs.box2d.car_racing import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from statistics import mean
from pyglet.window import key
import time
from tqdm import tqdm 
import os
from keras.utils import normalize

C:\Users\Piyush\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Function for generating data by playing
def start_playing(n_episodes):
    a = np.array( [0.0, 0.0, 0.0] )
    def key_press(k, mod):
        global restart
        if k==0xff0d: restart = True
        if k==key.LEFT:  a[0] = -1.0
        if k==key.RIGHT: a[0] = +1.0
        if k==key.UP:    a[1] = +1.0
        if k==key.DOWN:  a[2] = +0.8   
    def key_release(k, mod):
        if k==key.LEFT  and a[0]==-1.0: a[0] = 0
        if k==key.RIGHT and a[0]==+1.0: a[0] = 0
        if k==key.UP:    a[1] = 0
        if k==key.DOWN:  a[2] = 0
    
    env = CarRacing()
    env.render()
    env.viewer.window.on_key_press = key_press
    env.viewer.window.on_key_release = key_release
    
    for episode in range(n_episodes):
        prev_observation = env.reset()
        if episode==0:
            X = np.array(prev_observation.tolist())
            y = np.array([0.,0.,0.])
        restart = False
        while True:
            action = a
            observation, reward, done, info = env.step(action)
            X = np.vstack((X, prev_observation))
            y = np.vstack((y, action))
            prev_observation=observation
            env.render()
            if done or restart: break
        time.sleep(2)    
    np.save('train_X.npy', X)
    np.save('train_y.npy', y)    
    env.close()
    return X, y

In [3]:
# Playing the to create data
X, y=start_playing(20)

Track generation: 1121..1405 -> 284-tiles track
Track generation: 925..1162 -> 237-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1228..1538 -> 310-tiles track
Track generation: 1132..1419 -> 287-tiles track
Track generation: 1282..1607 -> 325-tiles track
Track generation: 1171..1472 -> 301-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1120..1404 -> 284-tiles track
Track generation: 1030..1291 -> 261-tiles track
Track generation: 1119..1403 -> 284-tiles track
Track generation: 1126..1412 -> 286-tiles track
Track generation: 1413..1770 -> 357-tiles track
Track generation: 1230..1541 -> 311-tiles track
Track generation: 1246..1562 -> 316-tiles track
Track generation: 1128..1414 -> 286-tiles track
Track generation: 1085..1365 -> 280-tiles track
Track generation: 1387..1738 -> 351-tiles track
Track generation: 992..1251 -> 259-tiles track
Track generation: 1029..1296 -> 267-tiles 

In [8]:
# Combining gameplay files
temp = np.load('train_data-549.4999999999591.npy')
tempX = [i[0] for i in temp]
tempy = [i[1] for i in temp]
X = np.array(tempX)
y = np.array(tempy)
for file in tqdm(os.listdir('Data/')):
    temp = np.load('Data/' + file)
    tempX = [i[0] for i in temp]
    tempy = [i[1] for i in temp]
    tempX = np.array(tempX)
    tempy = np.array(tempy)
    X  = np.concatenate([X, tempX], axis=0)
    y  = np.concatenate([y, tempy], axis=0)

np.save('X.npy', X)    

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:05<00:00,  1.60it/s]


In [5]:
# Converting output to proper OneHot array
Y=[]
for i in range(len(y)):  
    if np.array_equal(y[i], np.array([0,1,0])):
        Y.append(np.array([0,0,1,0,0]))
    elif np.array_equal(y[i], np.array([0,0,0.8])):
        Y.append(np.array([0,0,0,1,0]))
    elif np.array_equal(y[i], np.array([1,0,0])):
        Y.append(np.array([0,1,0,0,0]))
    elif np.array_equal(y[i], np.array([-1,0,0])):
        Y.append(np.array([1,0,0,0,0]))
    else:
        Y.append(np.array([0,0,0,0,1]))      
      
np.save('y_modified.npy', Y)    

In [34]:
# Loading the data
X = np.load('train_X.npy')
y = np.load('y_modified.npy')[::5].copy()
X = X.reshape(-1,5,96,96,3)
print(X.shape)
print(y.shape)

(4090, 5, 96, 96, 3)
(4090, 5)


In [35]:
# Variation in generated data
left=np.array([1,0,0,0,0])
right=np.array([0,1,0,0,0])
straight=np.array([0,0,1,0,0])
stop=np.array([0,0,0,1,0])
no_key=np.array([0,0,0,0,1])
l=0
r=0
s=0
b=0
nk=0

for key in y:
    if np.array_equal(key, left):
        l+=1
    elif np.array_equal(key, right):
        r+=1
    elif np.array_equal(key, straight):
        s+=1
    elif np.array_equal(key, stop):
        b+=1
    elif np.array_equal(key, no_key):
        nk+=1    
      
print(l,r,s,b,nk)    

546 259 864 38 2383


In [38]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (96,96,3)))
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(5, activation = "softmax"))

In [39]:
optimizer = Adam(lr=0.0001)
model.compile(optimizer = optimizer, loss = "categorical_crossentropy")

In [40]:
model.fit(X, y, epochs=10, validation_split=0.2, batch_size=600, verbose=1, shuffle=True)

ValueError: Error when checking input: expected conv2d_7_input to have 4 dimensions, but got array with shape (4090, 5, 96, 96, 3)

In [28]:
# Saving the Model
model.save('Models/first_modelv1_35_1_02_2_5')

In [ ]:
# Loading the Model
model.load('Models/first-model')

In [33]:
#scores = []
#choices = []
env = CarRacing()
for each_game in range(2):
    #score = 0
    #game_memory = []
    prev_obs = env.reset()
    for i in range(1000):
        env.render()
        out = model.predict(prev_obs.reshape(-1,96,96,3))[0]
        print(out)
        
        if out[0]<=0.35:
            out[0]=0  
        if out[1]<=0.1:
            out[1]=0
        if out[2]<=0.04:
            out[2]=0   
        if out[3]<=0.2:
            out[3]=0
        if out[4]<=3:
            out[4]=0    
        
        output = np.argmax(out)
        action = np.zeros(5)
        np.put(action, output, 1)
        #choices.append(action)
        if np.array_equal(action, np.array([0,0,1,0,0])):
            action=np.array([0,1,0])
        elif np.array_equal(action, np.array([0,0,0,1,0])):
            action=np.array([0,0,0.8])
        elif np.array_equal(action, np.array([0,1,0,0,0])):
            action=np.array([1,0,0])
        elif np.array_equal(action, np.array([1,0,0,0,0])):
            action=np.array([-1,0,0])
        else:
            action=np.array([0,0,0,0,0])
        
        new_observation, reward, done, info = env.step(action)
        print(action,"by nn")
        prev_obs = new_observation
        #game_memory.append([new_observation, action])
        #score+=reward
        if done: 
            break
    #scores.append(score)
    #print('Episode {} Score: {}'.format(each_game+1,score))
    env.close()
#print('Average Score:',sum(scores)/len(scores))

Track generation: 1192..1494 -> 302-tiles track
[2.8968486e-07 5.3606255e-15 3.3170141e-13 1.6568108e-14 9.9999976e-01]
[-1  0  0] by nn
[1.0419476e-01 3.7953031e-01 3.3812103e-04 3.1869936e-01 1.9723739e-01]
[1 0 0] by nn
[2.4381025e-02 1.9249251e-01 2.1922966e-05 1.1991161e-01 6.6319293e-01]
[1 0 0] by nn
[1.0489094e-01 5.6807417e-01 5.2494441e-05 1.6216585e-01 1.6481648e-01]
[1 0 0] by nn
[9.7706378e-02 6.0192877e-01 1.1015229e-04 1.4102553e-01 1.5922917e-01]
[1 0 0] by nn
[4.7358967e-02 4.5400861e-01 2.1239711e-04 1.3545059e-01 3.6296943e-01]
[1 0 0] by nn
[6.5426603e-02 3.9163935e-01 2.7292766e-04 2.2731251e-01 3.1534863e-01]
[1 0 0] by nn
[0.14390734 0.31669527 0.00048078 0.2641218  0.2747948 ]
[1 0 0] by nn
[0.3451325  0.23645774 0.00122292 0.22601631 0.19117048]
[1 0 0] by nn
[0.4121595  0.18347396 0.00199233 0.20609584 0.19627838]
[-1  0  0] by nn
[0.43716827 0.11585358 0.00360433 0.1835794  0.25979438]
[-1  0  0] by nn
[0.4496222  0.10902863 0.00569224 0.1854639  0.250193  ]


[0.82718515 0.02414417 0.02134289 0.02948639 0.09784137]
[-1  0  0] by nn
[0.7867806  0.03224451 0.02768643 0.03285968 0.1204288 ]
[-1  0  0] by nn
[0.7804488  0.03802771 0.03286721 0.03590561 0.11275058]
[-1  0  0] by nn
[0.79325694 0.03513377 0.03164356 0.03334774 0.10661795]
[-1  0  0] by nn
[0.727143   0.05443229 0.05402447 0.03850125 0.12589906]
[-1  0  0] by nn
[0.5723005  0.0919495  0.11562894 0.05518858 0.16493241]
[-1  0  0] by nn
[0.40834683 0.1381002  0.20890503 0.07074043 0.17390749]
[-1  0  0] by nn
[0.2497253  0.20973039 0.26567468 0.08577047 0.18909922]
[0 1 0] by nn
[0.10121426 0.25474    0.32719177 0.09836921 0.21848476]
[0 1 0] by nn
[0.06077895 0.38411322 0.23950054 0.09509743 0.22050989]
[1 0 0] by nn
[0.0572793  0.539257   0.1429849  0.0908691  0.16960977]
[1 0 0] by nn
[0.03351404 0.7338351  0.05351977 0.05821537 0.1209158 ]
[1 0 0] by nn
[0.01418951 0.8400001  0.01456116 0.04898545 0.08226381]
[1 0 0] by nn
[0.01096617 0.881465   0.00603671 0.04124696 0.06028521]

[0.84115916 0.02218362 0.01147519 0.02862569 0.09655634]
[-1  0  0] by nn
[0.851834   0.02849766 0.01469941 0.0264869  0.07848209]
[-1  0  0] by nn
[0.7152144  0.06081146 0.04931629 0.0460043  0.12865354]
[-1  0  0] by nn
[0.5248276  0.13085027 0.09895778 0.07408092 0.17128344]
[-1  0  0] by nn
[0.3153069  0.24352308 0.12609708 0.09658383 0.21848917]
[1 0 0] by nn
[0.10182885 0.40491962 0.15324317 0.10681736 0.23319107]
[1 0 0] by nn
[0.04817093 0.5909393  0.10804575 0.08754963 0.16529436]
[1 0 0] by nn
[0.03350236 0.706148   0.06095919 0.06388056 0.13550985]
[1 0 0] by nn
[0.01772057 0.80293906 0.02382891 0.04148684 0.11402454]
[1 0 0] by nn
[0.00850995 0.869359   0.00732624 0.03479474 0.08001009]
[1 0 0] by nn
[0.00762812 0.8958884  0.0038249  0.03029178 0.06236687]
[1 0 0] by nn
[0.00777845 0.8946008  0.00417098 0.04007528 0.05337456]
[1 0 0] by nn
[0.00768829 0.89269495 0.00225261 0.0363763  0.06098785]
[1 0 0] by nn
[0.00705422 0.87020254 0.00240869 0.04303729 0.07729726]
[1 0 0] 

[0.02754474 0.76542354 0.03621097 0.06009214 0.11072862]
[1 0 0] by nn
[0.02402456 0.7422504  0.05009776 0.06586714 0.11776012]
[1 0 0] by nn
[0.0315005  0.73003304 0.04026258 0.05143166 0.1467722 ]
[1 0 0] by nn
[0.0270307  0.6482356  0.05880344 0.060258   0.20567217]
[1 0 0] by nn
[0.03618339 0.59249747 0.04781854 0.06762951 0.25587112]
[1 0 0] by nn
[0.04039856 0.4794965  0.06138216 0.06883058 0.34989226]
[1 0 0] by nn
[0.03972564 0.3089653  0.07131188 0.0814556  0.49854156]
[1 0 0] by nn
[0.05283254 0.22140822 0.08302233 0.08949206 0.5532449 ]
[1 0 0] by nn
[0.06060195 0.15369447 0.0832867  0.0771141  0.6253028 ]
[1 0 0] by nn
[0.10198791 0.1062924  0.08659554 0.07619936 0.6289248 ]
[1 0 0] by nn
[0.19502145 0.0763581  0.05490138 0.08334458 0.59037447]
[0 1 0] by nn
[0.31789073 0.05363033 0.06344884 0.07319509 0.491835  ]
[0 1 0] by nn
[0.4825051  0.03365598 0.04316415 0.05995441 0.38072035]
[-1  0  0] by nn
[0.66481733 0.01755915 0.01162313 0.05733624 0.2486642 ]
[-1  0  0] by nn


[9.9303353e-01 2.2011155e-03 8.5087908e-05 4.6325973e-03 4.7738959e-05]
[-1  0  0] by nn
[9.8183525e-01 5.2579464e-03 5.3709879e-04 9.6197696e-03 2.7500032e-03]
[-1  0  0] by nn
[0.95105255 0.01119215 0.0015477  0.01559181 0.02061575]
[-1  0  0] by nn
[0.9173537  0.01585124 0.00234183 0.02045822 0.04399499]
[-1  0  0] by nn
[0.8944652  0.01694414 0.00334568 0.01968226 0.06556274]
[-1  0  0] by nn
[0.85843474 0.02447465 0.00735196 0.02463489 0.08510374]
[-1  0  0] by nn
[0.82787406 0.02732665 0.01221179 0.02583438 0.10675307]
[-1  0  0] by nn
[0.73764515 0.05179626 0.03219051 0.03831658 0.14005145]
[-1  0  0] by nn
[0.50152266 0.11657971 0.12790805 0.06590905 0.18808055]
[-1  0  0] by nn
[0.26568642 0.23053873 0.19965023 0.08891467 0.21520992]
[1 0 0] by nn
[0.0654693  0.3996204  0.20677054 0.10260997 0.22552969]
[1 0 0] by nn
[0.03689954 0.521874   0.14152822 0.09302907 0.20666915]
[1 0 0] by nn
[0.02608589 0.658105   0.07426128 0.06848828 0.17305948]
[1 0 0] by nn
[0.02819012 0.718835

...

[0.72344697 0.02382221 0.0319492  0.03718515 0.18359645]
[-1  0  0] by nn
[0.8174497  0.01703428 0.01286676 0.02239116 0.13025811]
[-1  0  0] by nn
[0.7596891  0.01697081 0.02049737 0.02957286 0.17326993]
[-1  0  0] by nn
[0.76590806 0.02011761 0.02289444 0.02512874 0.16595116]
[-1  0  0] by nn
[0.70456994 0.03089353 0.03712906 0.03151873 0.19588888]
[-1  0  0] by nn
[0.5673984  0.06203363 0.08407158 0.0545953  0.23190111]
[-1  0  0] by nn
[0.42793247 0.11093932 0.13914444 0.07324617 0.24873762]
[-1  0  0] by nn
[0.38303503 0.19081536 0.139822   0.08658821 0.19973943]
[-1  0  0] by nn
[0.20287022 0.35332522 0.1472255  0.10031497 0.19626413]
[1 0 0] by nn
[0.0711083  0.5205881  0.12628767 0.09819843 0.18381743]
[1 0 0] by nn
[0.03105849 0.6995954  0.06748158 0.07228057 0.12958387]
[1 0 0] by nn
[0.02400991 0.7767985  0.03309062 0.05212785 0.11397313]
[1 0 0] by nn
[0.01750245 0.8244192  0.01411546 0.03911807 0.10484494]
[1 0 0] by nn
[0.0100187  0.84799576 0.00645751 0.04008808 0.095439

[0.18591277 0.13733411 0.09288011 0.08654026 0.49733275]
[1 0 0] by nn
[0.25805056 0.09268933 0.09066884 0.08479437 0.47379687]
[0 1 0] by nn
[0.4360666  0.0687978  0.06159207 0.07192019 0.36162338]
[-1  0  0] by nn
[0.59673    0.04694941 0.03621713 0.05337042 0.26673308]
[-1  0  0] by nn
[0.6980641  0.03100078 0.01675434 0.05033357 0.20384727]
[-1  0  0] by nn
[0.81849885 0.01865171 0.00642133 0.03997373 0.1164544 ]
[-1  0  0] by nn
[0.879988   0.014392   0.00332329 0.0352335  0.06706318]
[-1  0  0] by nn
[0.9230463  0.01134374 0.00180688 0.03038497 0.03341817]
[-1  0  0] by nn
[0.84984934 0.01758689 0.00450227 0.0783745  0.04968692]
[-1  0  0] by nn
[9.5838916e-01 7.8355009e-03 7.0248096e-04 2.3389639e-02 9.6831499e-03]
[-1  0  0] by nn
[0.93925786 0.00768513 0.00103174 0.0358747  0.01615055]
[-1  0  0] by nn
[9.8280925e-01 4.2729797e-03 2.9265639e-04 1.1738089e-02 8.8717625e-04]
[-1  0  0] by nn
[9.9236345e-01 2.3354515e-03 5.8003177e-05 5.2229357e-03 2.0145284e-05]
[-1  0  0] by nn

[0.45308295 0.05038518 0.07053152 0.05542626 0.37057403]
[-1  0  0] by nn
[0.37723452 0.07010991 0.0703174  0.07132357 0.41101462]
[-1  0  0] by nn
[0.21535549 0.11501201 0.12958555 0.10672727 0.4333197 ]
[0 1 0] by nn
[0.1244773  0.15369044 0.13752446 0.09683371 0.48747408]
[1 0 0] by nn
[0.10150088 0.19261652 0.12949677 0.10198015 0.47440568]
[1 0 0] by nn
[0.11596823 0.2316206  0.09717543 0.10923235 0.44600338]
[1 0 0] by nn
[0.08290904 0.20104232 0.11314304 0.1313137  0.47159186]
[1 0 0] by nn
[0.11313245 0.1805864  0.0840795  0.11803607 0.5041656 ]
[1 0 0] by nn
[0.12812452 0.16895093 0.06560799 0.12694427 0.51037234]
[1 0 0] by nn
[0.16799693 0.1271251  0.06451833 0.10502595 0.5353337 ]
[1 0 0] by nn
[0.21143277 0.09038129 0.04794779 0.09315498 0.55708313]
[0 1 0] by nn
[0.365688   0.05428008 0.02787549 0.08469977 0.4674566 ]
[-1  0  0] by nn
[0.48459998 0.04288603 0.01534429 0.0686595  0.38851026]
[-1  0  0] by nn
[0.5478343  0.03870633 0.01212127 0.06123884 0.3400993 ]
[-1  0  

[0.00759032 0.6871692  0.00801786 0.0392134  0.25800923]
[1 0 0] by nn
[0.0123978  0.58722824 0.01625272 0.05085087 0.3332703 ]
[1 0 0] by nn
[0.02451281 0.42925566 0.04114597 0.05803251 0.4470531 ]
[1 0 0] by nn
[0.04051423 0.28500527 0.10015925 0.05971223 0.51460904]
[1 0 0] by nn
[0.05117432 0.17155375 0.16948415 0.06298265 0.54480517]
[1 0 0] by nn
[0.08594786 0.12325296 0.1961893  0.06591219 0.52869767]
[0 1 0] by nn
[0.18783408 0.08899008 0.16452925 0.05308796 0.5055586 ]
[0 1 0] by nn
[0.39090118 0.05440576 0.06728669 0.03939293 0.44801348]
[-1  0  0] by nn
[0.58904064 0.03441522 0.02851799 0.02580727 0.32221892]
[-1  0  0] by nn
[0.6385928  0.0278448  0.01547305 0.0233538  0.2947355 ]
[-1  0  0] by nn
[0.6792807  0.02894178 0.01156268 0.02359171 0.25662315]
[-1  0  0] by nn
[0.7390474  0.02413108 0.00827343 0.01919112 0.20935696]
[-1  0  0] by nn
[0.7286255  0.02412601 0.01308652 0.02526395 0.20889808]
[-1  0  0] by nn
[0.8051637  0.01842927 0.00461343 0.01610189 0.15569167]
[-

[0.03283611 0.5219391  0.10053611 0.11625629 0.2284324 ]
[1 0 0] by nn
[0.05003967 0.52987933 0.06612951 0.09591586 0.25803563]
[1 0 0] by nn
[0.04855232 0.43436757 0.07864065 0.10996021 0.32847923]
[1 0 0] by nn
[0.06073758 0.40375772 0.08211031 0.10878359 0.34461072]
[1 0 0] by nn
[0.07735823 0.30003342 0.09617432 0.10995832 0.41647574]
[1 0 0] by nn
[0.09549344 0.21102668 0.11245399 0.1165531  0.4644728 ]
[1 0 0] by nn
[0.14962631 0.13857217 0.14374167 0.09541285 0.472647  ]
[0 1 0] by nn
[0.25512847 0.09648112 0.13769498 0.08727495 0.4234205 ]
[0 1 0] by nn
[0.45004347 0.0641128  0.0682096  0.06090323 0.35673085]
[-1  0  0] by nn
[0.5434881  0.05198086 0.03196574 0.05610878 0.31645653]
[-1  0  0] by nn
[0.62175447 0.03560732 0.01675418 0.03876554 0.28711846]
[-1  0  0] by nn
[0.6482704  0.04020661 0.01657189 0.03366957 0.26128152]
[-1  0  0] by nn
[0.59121954 0.03584111 0.03118122 0.04143729 0.3003209 ]
[-1  0  0] by nn
[0.722718   0.0288036  0.01380263 0.02428263 0.21039316]
[-1  